# [Парсинг внешних данных](https://stepik.org/lesson/866758/)

Эта тетрадка поможет разобраться вам в основных моментах парсинга внешних данных на python, познакомиться с полезными функциями и трюками. Вместе с вами мы посмотрим на возможности основных библиотек и закрепим на примерах.

### Оглавление ноутбука
<img src="https://sun1.tele2-kz-almaty.userapi.com/impg/GdOKq9yqtvjM1-lk5DsbsvvO2_XDXQfEIYJgHQ/8h1dFFWKFis.jpg?size=1920x960&quality=96&sign=9e084c8d32884fcbcbbc0a44ef601bea&type=album" align="right" width="528" height="528" />
<br>

<p><font size="3" face="Arial" font-size="large">
<ul type="square"><li><a href="#1">Анализ URL запроса</a><ul><li><a href="#2">Типы запросов и ответов</a></li></ul></li><li><a href="#3">Парсинг HTML страничек</a><ul><li><a href="#4">lxml с поддержкой CSS селекторов</a></li><li><a href="#5">BeautifulSoup</a></li><li><a href="#6">Proxy</a></li><li><a href="#7">fake_useragent</a></li></ul></li>
<li><a href="#8">Практические задачи</a></li></ul></font></p>

##  Анализ URL запроса

<p id="1"></p>
Рассмотрим для начала структуру URL адреса, это важно! URL адрес имеет определенную структуру, которая включает:

- метод доступа к ресурсу, который также называется сетевым __протоколом__;
- __авторизацию доступа__;
- __хосты__ – DNS адрес, который указан как IP адрес;
- __порт__ – еще одна обязательная деталь, которая включается в сочетание с IP адресом;
- __трек__ – определяет информацию о методе получения доступа;
- __параметр__ – внутренние данные ресурса о файле.


<img src="https://sun9-65.userapi.com/impg/VNKZOAYMIumG8qJcm8R1GBUG5iRj1Y1HJ0OeqQ/hEk7cu3dhzI.jpg?size=2143x1126&quality=96&sign=5c59d6c509304ba62f218e21254ad525&type=album"></img>

Можно разобрать URL на составляющие.

In [1]:
from urllib.parse import urlparse, parse_qsl, parse_qs

url = "http://www.example.com:80/path/to/myfile.html?key1=value1&key2=value2#SomewhereInTheDocument"
url_parsed = urlparse(url)
url_parsed

ParseResult(scheme='http', netloc='www.example.com:80', path='/path/to/myfile.html', params='', query='key1=value1&key2=value2', fragment='SomewhereInTheDocument')

In [2]:
url_parsed.path

'/path/to/myfile.html'

Парсинг самого запроса внутри URL.

In [3]:
dict(parse_qsl(url_parsed.query))

{'key1': 'value1', 'key2': 'value2'}

### Типы запросов и ответов

<p id="2"></p>
C ссылками разобрались. Что же представляет из себя один из основных протоколов интернета? Тип HTTP-запроса (также называемый HTTP-метод) указывает серверу на то, какое действие мы хотим произвести с ресурсом. Изначально (в начале 90-х) предполагалось, что клиент может хотеть от ресурса только одно — получить его, однако сейчас по протоколу HTTP можно создавать посты, редактировать профиль, удалять сообщения и многое другое. И эти действия сложно объединить термином «получение».

Для разграничения действий с ресурсами на уровне HTTP-методов и были придуманы следующие варианты:  <img src="https://sun9-23.userapi.com/impg/ieMiTgQNwhi-RnoMCI24CzNn0fC7dMi8HO7QSA/CBgxOLofBqc.jpg?size=509x252&quality=96&sign=08a9cc2b888fd21ebdb9852fb3e30868&type=album" align="right" padding=100px></img>
<br> 
- GET — получение ресурса 
- POST — создание ресурса
- PUT — обновление ресурса
- DELETE — удаление ресурса 

Обратите внимание на тот факт, что спецификация HTTP не обязывает сервер понимать все методы (которых на самом деле гораздо больше, чем 4) — обязателен только GET, а также не указывает серверу, что он должен делать при получении запроса с тем или иным методом. А это значит, что сервер в ответ на запрос DELETE /index.php HTTP/1.1 не обязан удалять страницу index.php на сервере, так же как на запрос GET /index.php HTTP/1.1 не обязан возвращать вам страницу index.php, он может ее удалять.


Рассмотрим что здесь все-таки происходит на практике.

In [35]:
import requests

Попробуем отправить и записать сообщение на сервисе [transfer.sh](https://www.transfer.sh). Сервис позволяет пользователям хранить, синхронизировать и обмениваться файлами в Интернете с другими пользователями прямо из терминала.


In [43]:
r_put = requests.put('https://transfer.sh/arg.txt', data='Bonjour le monde!')
r_put

<Response [200]>

In [44]:
r_put.ok, r_put.status_code

(True, 200)

In [45]:
print(r_put.text)

https://transfer.sh/dPN7dG/arg.txt


Видим, что все ок.

Рассмотрим метод создание запроса POST. Также пытаемся отправить запрос на запись текста. Укажем URL интересующего нас ресурса и дополнительные параметры в виде словаря.

In [48]:
page = requests.post('https://controlc.com/index.php?act=submit', data={
    'subdomain': '',
    'antispam': 1,
    'website': '',
    'paste_title': 'Заметка',
    'input_text': 'Привет!',
    'timestamp': 'ba68753935524ba7096650590c86633b',
    'paste_password': '',
    'code': 0,
}, headers={'accept-encoding': 'identity', 'referer': 'https://controlc.com/'})
page

<Response [200]>

Теперь попробуем передать файл с помощью POST запроса.

In [52]:
fs = {"pdb_file": open('./data/PC.pdf','rb')}
d = {"method": 'Structure'}
req = requests.post("http://www.sbg.bio.ic.ac.uk/~suspect/submit.cgi", files=fs, data=d)
req.url

'http://www.sbg.bio.ic.ac.uk/~suspect/jobmonitor.cgi?jobid=393d065075278d5c'

## Парсинг HTML страничек

<p id="3"></p>
Для понимания парсинга нужно знать структуру HTML. Как театр начинается с вешалки, так и любой HTML-документ начинается с базовой структуры. Она включает в себя теги, которые есть в любом HTML-файле. Эти теги и служебная информация нужны браузеру для корректного отображения информации.


<img src="https://dev-gang.ru/static/storage/319901168311547457028755251266243086714.gif"></img>

<div class="alert alert-info">
    
Тег `<html></html>` является основой основ. Именно внутри него располагается вся информация. Благодаря этому тегу браузер понимает, где начинается контент, который необходимо обработать как HTML. Тег `<head></head>` служит для хранения служебной информации. Здесь возможны самые разные сочетания тегов, которые подсказывают браузеру название страницы, описание, ключевые слова и так далее. Такая информация называется метаинформацией. После тега `<head>` в документе указывается парный тег `<body></body>`, который является «телом» всей страницы. Именно здесь размещается вся информация, которая будет выведена на странице.  
</div>
<img  align="right" width='500px' src="https://q-bit.biz/uploads/article/Tegs_1539003896.png"></img>


Внутри `<body></body>` находится вся структура документа состоящая из тегов, атрибутов и материалов. Рассмотрим единичную структуру - тег: 
<br>Весь текст, заключённый между начальным и конечным тегом, включая <br> и сами эти теги, называется элементом. Сам же текст между тегами — содержанием элемента. Содержание элемента может включать в себя любой текст, в том числе и другие элементы. У тега могут быть свойства, называемые атрибутами, дающие дополнительные возможности форматирования текста. Они записываются в виде сочетания: имя атрибута-значения, причём текстовые значения заключаются в кавычки.

###  lxml с поддержкой CSS селекторов

<p id="4"></p>
Так как html (и xml) имеют древовидную структуру, до любого элемента всегда существет единственный путь, XPath.

In [60]:
!pip install cssselect

Попробуем поиграться со страницей фильма "Стражи Галактики". Допустим у нас соревнование по предсказанию жанра фильма и было принято решение о парсинге дополнительных данных. Приступим!

In [61]:
from lxml import etree, html as lhtml

In [65]:
tree = lhtml.fromstring(open('data/689066_2.html', 'r', encoding='utf-8').read())

С помощью `xpath` узнаем информацию о фильме. Сначала указывается интересующий нас тег, допустим `//div`, а затем его атрибут. Настоятельно рекомендуем на этих пунктах открыть исходный код странички и самостоятельно убедиться что откуда берется. Это не сложно :)

In [66]:
film_info = {
    'title': tree.xpath('//h1[@itemprop="name"]/span/text()')[0],
    'title-original': tree.xpath('//span[starts-with(@class, "styles_originalTitle__")]')[0].text,
    'rating': float(tree.cssselect('a.film-rating-value')[0].text),   # поддержка CSS-селекторов
    'desription': '\n'.join(tree.xpath('//div[starts-with(@class, "styles_synopsisSection")]//text()'))
}

film_info

{'title': 'Стражи Галактики',
 'title-original': 'Guardians of the Galaxy',
 'rating': 7.763,
 'desription': 'Отважному путешественнику Питеру Квиллу попадает в руки таинственный артефакт, принадлежащий могущественному и безжалостному злодею Ронану, строящему коварные планы по захвату Вселенной. Питер оказывается в центре межгалактической охоты, где жертва — он сам.\nЕдинственный способ спасти свою жизнь — объединиться с четверкой нелюдимых изгоев: воинственным енотом по кличке Ракета, человекоподобным деревом Грутом, смертельно опасной Гаморой и одержимым жаждой мести Драксом, также известным как Разрушитель. Когда Квилл понимает, какой силой обладает украденный артефакт и какую опасность он представляет для вселенной, одиночка пойдет на все, чтобы сплотить случайных союзников для решающей битвы за судьбу галактики.'}

Далее найдем ссылку на просмотр. Ссылочки практически всегда находятся в тегах `<a href="your_link"></a>`.

In [67]:
watch = tree.xpath('//a[contains(@class, "kinopoisk-watch-online-button")]/attribute::href')
film_info['watch'] = watch
film_info

{'title': 'Стражи Галактики',
 'title-original': 'Guardians of the Galaxy',
 'rating': 7.763,
 'desription': 'Отважному путешественнику Питеру Квиллу попадает в руки таинственный артефакт, принадлежащий могущественному и безжалостному злодею Ронану, строящему коварные планы по захвату Вселенной. Питер оказывается в центре межгалактической охоты, где жертва — он сам.\nЕдинственный способ спасти свою жизнь — объединиться с четверкой нелюдимых изгоев: воинственным енотом по кличке Ракета, человекоподобным деревом Грутом, смертельно опасной Гаморой и одержимым жаждой мести Драксом, также известным как Разрушитель. Когда Квилл понимает, какой силой обладает украденный артефакт и какую опасность он представляет для вселенной, одиночка пойдет на все, чтобы сплотить случайных союзников для решающей битвы за судьбу галактики.',
 'watch': ['https://hd.kinopoisk.ru/film/4a297ba39cb704fa9a81855f76ab1d73?from=button_online&watch=']}

Найдем картинку аналогичным образом. Тег `<img>`.

In [16]:
image = tree.xpath('//img[contains(@class, "film-poster")]//attribute::srcset')
image

['//avatars.mds.yandex.net/get-kinopoisk-image/1773646/2e6ab20b-7cf1-49e7-b465-bd5a71c13fa3/300x450 1x, //avatars.mds.yandex.net/get-kinopoisk-image/1773646/2e6ab20b-7cf1-49e7-b465-bd5a71c13fa3/600x900 2x']

###  BeautifulSoup

<p id="5"></p>
С простым парсингом разобрались, перейдем к более интересной и обширной библиотеке <a href = "https://www.crummy.com/software/BeautifulSoup/bs4/doc/">Beautifulsoup</a>. Здесь есть множество полезных алгоритмов упрощающих вашу работу, опимизирующих поиск множества информации и т.д. Будем работать с той же страничкой для удобства. Напоминаем, что в ходе этой практики не лишним будет открыть исходный код странички и самостоятельно убедиться что откуда берется, это очень поможет быстро разобраться.

<img src="https://sun9-58.userapi.com/impg/o_qPHMIWsPUG9VLR3vbjjs6fap-iX89MUHkdWw/oOMz3ZM20QY.jpg?size=751x335&quality=96&sign=5358d071a6a30677ff578a72c2a4fc2e&type=album"></img>

In [17]:
from bs4 import BeautifulSoup
import pandas as pd

In [18]:
soup = BeautifulSoup(open('data/689066_2.html', 'rb').read(), 'lxml')

In [19]:
from operator import attrgetter, itemgetter

Берем основную информацию со странички. Запустив сессию `BeautifulSoup`, найдем теги, здесь это можно гораздо проще и приятнее для глаз - `.find("tag", attribute="value")`. Для того чтобы найти все встречающиеся варианты используем `.find_all("tag")`

In [20]:
desc = soup.find('div', itemprop='description')
desc = soup.find('div', class_=lambda s: s and s.startswith("styles_synopsisSection")).find_all('p')

film_info = {
    'title': soup.find('h1', itemprop='name').find('span').text,
    'title-original': soup.find('span', class_=lambda s: s and s.startswith('styles_originalTitle__')).text,
    'rating': float(soup.select_one('a.film-rating-value').text), 
    'description': '\n'.join(map(attrgetter('text'), desc))
}
film_info

{'title': 'Стражи Галактики',
 'title-original': 'Guardians of the Galaxy',
 'rating': 7.763,
 'description': 'Отважному путешественнику Питеру Квиллу попадает в руки таинственный артефакт, принадлежащий могущественному и безжалостному злодею Ронану, строящему коварные планы по захвату Вселенной. Питер оказывается в центре межгалактической охоты, где жертва — он сам.\nЕдинственный способ спасти свою жизнь — объединиться с четверкой нелюдимых изгоев: воинственным енотом по кличке Ракета, человекоподобным деревом Грутом, смертельно опасной Гаморой и одержимым жаждой мести Драксом, также известным как Разрушитель. Когда Квилл понимает, какой силой обладает украденный артефакт и какую опасность он представляет для вселенной, одиночка пойдет на все, чтобы сплотить случайных союзников для решающей битвы за судьбу галактики.'}

Скачиваем данные, чтобы сделать таблицу.

In [21]:
header = soup.find('h3', class_="film-page-section-title")
table = header.next_sibling
rows = table.find_all('div', recursive=False)

len(rows)

23

Так можно наглядно убедиться что у нас с данными и начать с ними, если нужно, необходмую работу.

In [22]:
data = []

for row in rows:
    cols = map(lambda x: x.text, row.find_all('div'))
    data.append(cols)

data = pd.DataFrame(data)
data

,0,1,2
0,Год производства,2014,None
1,Страна,США,None
2,Жанр,"фантастика, боевик, приключения, комедияслова","фантастика, боевик, приключения, комедия"
3,Слоган,"«Мстители спасают лишь Землю бренную, а эти ре...","«Мстители спасают лишь Землю бренную, а эти ре..."
4,Режиссер,Джеймс Ганн,None
5,Сценарий,"Джеймс Ганн, Николь Перлман, Дэн Абнетт, ...",None
6,Продюсер,"Кевин Файги, Виктория Алонсо, Джэми Кристофер,...",None
7,Оператор,Бен Дэвис,None
8,Композитор,Тайлер Бейтс,None
9,Художник,"Чарльз Вуд, Рави Бансал, Мэттью Бродерик, ...",None


Найдем картинки. Как было сказано выше, в данном случае использовать удобнее `.find_all("tag")`.

In [23]:
soup = BeautifulSoup(open('data/689066_stills_2.html', 'rb').read(), 'html.parser')
list(map(lambda s: s.attrs['src'], soup.find('table', class_='fotos').find_all("img")))[:10]

['https://st.kp.yandex.net/images/kadr/sm_2802088.jpg',
 'https://st.kp.yandex.net/images/kadr/sm_2802087.jpg',
 'https://st.kp.yandex.net/images/kadr/sm_2802086.jpg',
 'https://st.kp.yandex.net/images/kadr/sm_2802085.jpg',
 'https://st.kp.yandex.net/images/kadr/sm_2802084.jpg',
 'https://st.kp.yandex.net/images/kadr/sm_2802083.jpg',
 'https://st.kp.yandex.net/images/kadr/sm_2802082.jpg',
 'https://st.kp.yandex.net/images/kadr/sm_2802081.jpg',
 'https://st.kp.yandex.net/images/kadr/sm_2751304.jpg',
 'https://st.kp.yandex.net/images/kadr/sm_2485201.jpg']

##  Proxy

<p id="6">Прокси-сервер (сервер-посредник) — промежуточный сервер (комплекс программ) в компьютерных сетях, выполняющий роль посредника между пользователем и целевым сервером (при этом о посредничестве могут как знать, так и не знать обе стороны), позволяющий клиентам как выполнять косвенные запросы (принимая и передавая их через прокси-сервер) к другим сетевым службам, так и получать ответы. Посмотрим на свои параметры.</p>

In [69]:
def browser_stats_from_yandex(**params):
    page = requests.get('https://yandex.ru/internet/', **params)
    soup = BeautifulSoup(page.content, 'html.parser')

    params = {}
    for e in soup.find('ul', class_='general-info').find_all('li'):
        key = e.find('h3').text
        val = e.find('div', {'class': None})
        val = val.text if val else '–'
        params[key] = val
    return params

Здесь можем посмотреть на свои исходные параметры.

In [70]:
browser_stats_from_yandex()

{'IPv4-адрес': '85.117.100.138',
 'IPv6-адрес': '–',
 'Браузер': 'Неизвестный браузер',
 'Разрешение экрана': '–',
 'Регион': '–',
 'JavaScript отключен': 'В вашем браузере отсутствует или выключена поддержка JavaScript.'}

А теперь попробуем натянуть прокси.

In [71]:
proxies = { 'https://': '85.25.91.156:5000', }

browser_stats_from_yandex(proxies=proxies)

{'IPv4-адрес': '85.117.100.138',
 'IPv6-адрес': '–',
 'Браузер': 'Неизвестный браузер',
 'Разрешение экрана': '–',
 'Регион': '–',
 'JavaScript отключен': 'В вашем браузере отсутствует или выключена поддержка JavaScript.'}

Также можно менять Браузер и другие параметры.

In [72]:
browser_stats_from_yandex(proxies=proxies, headers={'User-Agent': 'TwitterBot'})

{'IPv4-адрес': '85.117.100.138',
 'IPv6-адрес': '–',
 'Браузер': 'Twitterbot',
 'Разрешение экрана': '–',
 'Регион': '–',
 'JavaScript отключен': 'В вашем браузере отсутствует или выключена поддержка JavaScript.'}

# БОНУС

<p id="7">Пакет fake_useragent позволяет создавать виртуальную сессию любого браузера и параметров. </p>

In [30]:
!pip install fake_useragent

     |████████████████████████████████| 50 kB 253 kB/s            
Note: you may need to restart the kernel to use updated packages.


Если вы хотите указать свой желаемый браузер, вы можете сделать это с помощью аргумента браузера (по умолчанию: `["chrome", "edge", "internet explorer", "firefox", "safari", "opera"]`).

In [31]:
from fake_useragent import UserAgent
   

ua = UserAgent()
print(ua.chrome)
header = {'User-Agent':str(ua.chrome)}
print(header)
url = "https://www.kaggle.com/"
htmlContent = requests.get(url, headers=header)
print(htmlContent)

Mozilla/5.0 (X11; U; Linux x86_64; en-US) AppleWebKit/532.0 (KHTML, like Gecko) Chrome/4.0.212.0 Safari/532.0
{'User-Agent': 'Mozilla/5.0 (Windows; U; Windows NT 5.2; en-US) AppleWebKit/531.3 (KHTML, like Gecko) Chrome/3.0.193.2 Safari/531.3'}
<Response [200]>


Всю информацию об этой интересной библиотеке можно найти [здесь](https://github.com/fake-useragent/fake-useragent).

<p id="8"></p>

# Практические задачи

## Задача 1.

Напишем свой небольшой парсер. Попробуем спарсить информацию с довольно простого сайта керамических изделий. Допустим у нас соревнование в ритейле и нам зачем-то понадобились данные по керамике. Будем использовать рассмотренную библиотеку BeautifulSoup. Ссылка на исходную страницу для создания сессии  - [здесь](https://www.borkeramika.ru/posuda-copy/). Задача состоит в том, чтобы найти все элементы с категориями и занести ссылки на них в массив `categories`, далее итеративно для каждой категории создаем сессию BeautifulSoup, и находим все ссылки на товары, делаем общий массив для всех товаров всех категорий `result`, его возвращаем. 

<img src="https://sun9-76.userapi.com/impg/n7iM4Y3j7C0cfJE661O71_2ntuItRcXNbvCYEA/MMkAB2qUet0.jpg?size=2066x1438&quality=96&sign=fdb120833b8b90d14e71e5fd1db1d4e7&type=album"></img>

_Замечание_: Для множественного поиска не забываем использовать `.find_all("your_tag")`, не забываем проверить дублирование ссылок из разных категорий.


__На вход подается:__

[Ссылка на интересующий нас сайт](https://www.borkeramika.ru/posuda-copy/)

__На выходе принимается:__

Список `result` со ссылками на все товары сайта.



In [74]:
#your code

## Задача 2.

Используем результаты первой задачи. Попробуем сделать датасет товаров с интересующими нас фичами. Итеративно обходим все товары по добытым ссылкам из первой задачи. На странице товара находим нужные нам элементы:

1. Название товара
2. Цена
3. Артикул
4. Описание
5. Таблицу характеристик

Заходя на страницу товара складываем данные в словарь `item_dict` с ключами `"name", "price", "vendor_code", "description", "properties_table"`. По ключу `"properties_table"` кладем все имеющиеся характеристики в словарь, где ключ - название характеристики, значение - ее величина/описание. Далее требуется занести все словари в общий список `all_items`.

<img src="https://sun9-37.userapi.com/impg/ScDyisOPZD7-IB44oD1zhV5s0FWdZ75xxZSn0w/CuBBGkBqexQ.jpg?size=2440x1370&quality=96&sign=ed7ccec87bf5ff83101b309764c3736a&type=album"></img>

_Замечание:_ Зачастую товаров бывают разные, не везде есть те или иные характеристики. Важно!! Если нет пункта 1 - Название товара или пункта 2 - Цена, то такие товары нас не интересуют, их не заносим. В таких случаях либо отстуствует текст тега, либо BeautifulSoup выдаст `AttributeError`. Стоит помнить о конструкции `try-except`.

__На вход подается:__

Список `result` со ссылками на все товары сайта из первой задачи.


__На выходе принимается:__

Список `all_items` со словарями данных о товарах.

In [75]:
#your code